In [1]:
#This crops and animates videos for traditional YouTube 1920x1080

In [2]:
from moviepy.audio.fx.all import *
from moviepy.editor import *
from moviepy.video.fx.all import *

from skimage.filters import gaussian

In [3]:
#Defining the video file path
video_file_path = '/home/rohit/Videos/sasha_clips/gi_e1.mp4'

In [4]:
#Defining the starting ane end clips
t_sub_start = (0, 0, 26.500)
t_sub_end = (0, 11, 22.000)

In [5]:
#Extracting original subclip
main_clip_original = VideoFileClip(video_file_path).subclip(t_sub_start, t_sub_end)
#main_clip_original.write_videofile('/home/rohit/Videos/sasha_clips/test.mp4', logger=None)

In [6]:
#Adding logo and instahandle
logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_symbol_only/symbol3.png")
       .set_duration(main_clip_original.duration)
       .set_position((1500, -40))
       .resize(0.25)
       .crossfadeout(0.3))

#Original
#set_position((-35, -40))

handle = (TextClip('SASHA COBRA', font='Montserrat-SemiBold', fontsize=25, color='white', 
                   align='center', kerning=5)
          .set_position((1660, 52))
          .set_duration(main_clip_original.duration)
          .crossfadeout(0.3))

#Original
#set_position((125, -40))

In [7]:
main_clip_logo = CompositeVideoClip([main_clip_original, logo, handle])
#main_clip_logo.ipython_display(width=640, t=5)
#main_clip_logo.save_frame('/home/rohit/Downloads/logo_1920x1080_test.png', t=5)

In [8]:
#Defining a function to blur a frame
def blur(image):
    
    """
    Returns a blurred image using gaussian filter
    with radius
    """
    
    return gaussian(image.astype(float), sigma=7)

In [9]:
#Defining a function to generate timestamps for subclips
#Takes into account minute and seconds
def timestamp_gen(start, increment):
    """
    start - start time tuple (hours, min, seconds)
    increment - in seconds
    """
    
    #Setting stop tuple to be same as start
    stop = start
    
    #Converting to a list to increment as tuple is immutable
    temp_list = list(stop)
    
    #Adding length of post clip animation duration to the end of the main clip
    new_min = temp_list[2] + increment
    
    #Updating the list based on whether the incremented minute
    #is greater than or less than 60min as the hour has to be 
    #accounted for.
    if new_min < 60:
        temp_list[2] = new_min
    elif new_min > 60:
        temp_list[1] += 1
        temp_list[2] = new_min-60
        
    #Converting list back to tuple
    stop = tuple(temp_list)
    
    return stop

In [10]:
#Adding post clip animation
def add_post_clip_ani(main_clip_end_time):
    
    #Defining how long we want the post clip animation to be
    #This value comes from correctly cutting the music at the 
    #beginning of the original file.
    post_clip_duration = 9.51 #seconds
    
    #Computing time steps for the complete clip
    post_clip_start = main_clip_end_time
    post_clip_end = timestamp_gen(post_clip_start, post_clip_duration)
    
    #Extracting the subclip
    post_clip = VideoFileClip('/home/rohit/Videos/sasha_clips/gi_e1.mp4').subclip(post_clip_start, post_clip_end)
    
    #Muting audio in post clip
    post_clip_mute = post_clip.volumex(0)
    
    #Defining the subclip durations of the post clip
    #We have 4 subclips based on the "eh" sound in the music
    
    #Defining the timestamps in a dictionary
    pc_t = {}
    
    pc_t['1_start'] = (0, 0, 0.0)
    pc_t['1_end'] = (0, 0, 1.111)
    pc_t['2_start'] = pc_t['1_end']
    pc_t['2_end'] = (0, 0, 3.544)
    pc_t['3_start'] = pc_t['2_end']
    pc_t['3_end'] = (0, 0, 6.040)
    pc_t['4_start'] = pc_t['3_end']
    pc_t['4_end'] = (0, 0, 8.47)
    pc_t['5_start'] = pc_t['4_end']
    pc_t['5_end'] = (0, 0, post_clip.duration)
    
    #Extracting the subclips corresponding to the subclip stamps
    #also applying blur after extraction
    #storing in a dictionary
    pc = {}
    for i in range(1, 6):
        pc[i] = post_clip_mute.subclip(pc_t[str(i)+'_start'], pc_t[str(i)+'_end']) 
        pc[i] = pc[i].fl_image(blur)
    
    #Defining text clips
    getting_intimate_1 = (TextClip('GETTING INTIMATE', font='Montserrat-SemiBold', fontsize=70, color='white',
                                align='center', kerning=10)
                          .set_position('center', 'center')
                          .set_duration(pc[1].duration)
                          .crossfadein(0.3))
    
    getting_intimate_2 = (TextClip('GETTING INTIMATE', font='Montserrat-SemiBold', fontsize=70, color='white',
                                align='center', kerning=10)
                          .set_position('center', 'center')
                          .set_duration(pc[2].duration)
                          .crossfadeout(0.3))
    
    with_sc = (TextClip('WITH SASHA COBRA', font='Montserrat-SemiBold', fontsize=25, color='white', align='center')
                        .set_position((0.577, 0.54), relative=True)
                        .set_duration(pc[2].duration)
                        .crossfadein(0.3)
                        .crossfadeout(0.3))
    
    watch_fv = (TextClip('Watch full video series', font='Montserrat-SemiBold', 
                        fontsize=35, color='white', align='center')
                        .set_position(('center', 'center'))
                        .set_duration(pc[3].duration)
                        .crossfadein(0.3)
                        .crossfadeout(0.3))
    
    at_sc = (TextClip('at sashacobra.com', font='Montserrat-SemiBold',
                      fontsize=35, color='white', align='center')
                      .set_position(('center', 0.53), relative=True)
                      .set_duration(pc[3].duration)
                      .crossfadein(0.3)
                      .crossfadeout(0.3))
    
    #Defining final logo image clip
    final_logo_4 = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_21.png")
                    .set_duration(pc[4].duration)
                    .set_pos(("center", "center"))
                    .resize(0.7)
                    .crossfadein(0.3))
    
    final_logo_5 = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_21.png")
                    .set_duration(pc[5].duration)
                    .set_pos(("center", "center"))
                    .resize(0.7)
                    .crossfadeout(pc[5].duration))
    
    #Generating composite video clips
    comp1 = CompositeVideoClip([pc[1], getting_intimate_1])
    comp2 = CompositeVideoClip([pc[2], getting_intimate_2, with_sc])
    comp3 = CompositeVideoClip([pc[3], watch_fv, at_sc])
    comp4 = CompositeVideoClip([pc[4], final_logo_4])
    comp5 = CompositeVideoClip([pc[5], final_logo_5])
    
    post_clip_with_text = concatenate_videoclips([comp1, comp2, comp3, comp4, comp5])
    
    #Setting post_clip audio

    #Importing audio file
    post_clip_audio = AudioFileClip('/home/rohit/Videos/sasha_clips/temp/intro_audio.wav')
    
    #Setting audio
    post_clip_final = post_clip_with_text.set_audio(post_clip_audio)
    
    #Audio fade in and out effects
    post_clip_final = post_clip_final.audio_fadein(0.3)
    post_clip_final = post_clip_final.audio_fadeout(pc[5].duration)
    
    #post_clip_final.write_videofile('/home/rohit/Videos/sasha_clips/post_clip_ani_vertical.mp4', logger=None)
    
    return post_clip_final

In [11]:
post_final_clip = add_post_clip_ani(t_sub_end)

In [12]:
#post_final_clip.write_videofile('/home/rohit/Videos/sasha_clips/post_clip_ani.mp4')
#post_final_clip.ipython_display(width=640, t=3)

In [13]:
final_export = concatenate_videoclips([main_clip_logo, post_final_clip], method='chain')

In [14]:
#Exporting
export_path = '/home/rohit/Videos/sasha_clips/sashas_opinion_love.mp4'
final_export.write_videofile(export_path)

chunk:   0%|          | 0/14664 [00:00<?, ?it/s, now=None]

Moviepy - Building video /home/rohit/Videos/sasha_clips/sashas_opinion_love.mp4.
MoviePy - Writing audio in sashas_opinion_loveTEMP_MPY_wvf_snd.mp3


t:   0%|          | 3/15945 [00:00<13:32, 19.63it/s, now=None]         

MoviePy - Done.
Moviepy - Writing video /home/rohit/Videos/sasha_clips/sashas_opinion_love.mp4



Moviepy - Done !
Moviepy - video ready /home/rohit/Videos/sasha_clips/sashas_opinion_love.mp4
